In [38]:
import numpy as np
import os
import sys
sys.path.append('../')
import cv2

In [2]:
from utils import load_datasets

In [14]:
def load_datasets(datasets_dir, multi_class=True, read_from_stubs=False, save_stub=False):
    """Given a directory containing all our datasets, return our train and validation data in 2 numpy array.

    Args:
        datasets_dir (str): path/to/dataset_dir
        multi_class (bool, optional): Choose if you do multi-class or binary classification. Defaults to True.
        read_from_stubs (bool, optional): Read data from npy file (faster). Defaults to False.
        save_stub (bool, optional): Save a npy file. Defaults to False.

    Returns:
        np.array: train and validation datasets in np.array
    """
    if read_from_stubs and (os.path.exists(os.path.join(datasets_dir, 'dataset_01.npy')) and os.path.exists(os.path.join(datasets_dir, 'dataset_02.npy'))):
        with open(os.path.join(datasets_dir, 'dataset_01.npy'), 'rb') as f:
            dataset_01 = np.load(f)
        with open(os.path.join(datasets_dir, 'dataset_02.npy'), 'rb') as f:
            dataset_02 = np.load(f)
        
        return dataset_01, dataset_02
            
            
    train_total_size = 0
    val_total_size = 0
    
    if multi_class:
        for folder in os.listdir(datasets_dir):
            if folder.endswith('01'):
                train_total_size  += len(os.listdir(os.path.join(datasets_dir, folder)))
            elif folder.endswith('02'):
                val_total_size += len(os.listdir(os.path.join(datasets_dir, folder)))  
    
    else:
        for folder in os.listdir(datasets_dir)[:4]:
            if folder.endswith('01'):
                train_total_size  += len(os.listdir(os.path.join(datasets_dir, folder)))
            elif folder.endswith('02'):
                val_total_size += len(os.listdir(os.path.join(datasets_dir, folder)))
        
    dataset_01 = np.empty(shape=(train_total_size, 540, 920, 3), dtype=np.float32)
    dataset_02 = np.empty(shape=(val_total_size, 540, 920, 3), dtype=np.float32)
        
    i = 0
    for folder in os.listdir(datasets_dir):
        if folder.endswith('01'):
            for file in os.listdir(os.path.join(datasets_dir, folder)):
                dataset_01[i] = cv2.imread(os.path.join(datasets_dir, folder, file)) / 255.0
                i += 1
    i = 0
    for folder in os.listdir(datasets_dir):
        if folder.endswith('02'):
            for file in os.listdir(os.path.join(datasets_dir, folder)):
                dataset_02[i] = cv2.imread(os.path.join(datasets_dir, folder, file)) / 255.0
                i += 1
    
    if save_stub:
        with open(os.path.join(datasets_dir, 'dataset_01.npy'), 'wb') as f:
            np.save(f, dataset_01)
        with open(os.path.join(datasets_dir, 'dataset_02.npy'), 'wb') as f:
            np.save(f, dataset_02)

    return dataset_01, dataset_02

In [55]:
datasets_dir = '../datasets/Movies'
multi_class = False

In [33]:
train_total_size = 0
val_total_size = 0
multi_class = False

if multi_class:
        for folder in os.listdir(datasets_dir):
            if folder.endswith('01'):
                train_total_size  += len(os.listdir(os.path.join(datasets_dir, folder)))
            elif folder.endswith('02'):
                val_total_size += len(os.listdir(os.path.join(datasets_dir, folder)))      
else:
    for folder in os.listdir(datasets_dir)[:4]:
        if folder.endswith('01'):
            train_total_size  += len(os.listdir(os.path.join(datasets_dir, folder)))
        elif folder.endswith('02'):
            val_total_size += len(os.listdir(os.path.join(datasets_dir, folder)))

In [34]:
print(int(0.8*train_total_size))
print(int(0.2*train_total_size))

800
200


In [35]:
dataset_01_train = np.empty(shape=(int(0.8*train_total_size), 540, 920, 3), dtype=np.float32)
dataset_01_test = np.empty(shape=(int(0.2*train_total_size), 540, 920, 3), dtype=np.float32)
dataset_02 = np.empty(shape=(val_total_size, 540, 920, 3), dtype=np.float32)

In [54]:
for folder in os.listdir(datasets_dir)[:4]:
    idx_train = 0
    idx_test = 0
    if folder.endswith('01'):
        len_folder = len(os.listdir(os.path.join(datasets_dir, folder)))
        for i, file in enumerate(os.listdir(os.path.join(datasets_dir, folder))[:int(0.8*len_folder)]):
            print(i) 
        idx_train += len()
        for i, file in enumerate(os.listdir(os.path.join(datasets_dir, folder))[:int(0.2*len_folder)]):
            print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [37]:
i = 0
for folder in os.listdir(datasets_dir)[:4]:
    if folder.endswith('01'):
        for file in os.listdir(os.path.join(datasets_dir, folder)):
            if i < int((0.8*train_total_size/2)):
                #print(f'train : {i}')
                dataset_01_train[i] = cv2.imread(os.path.join(datasets_dir, folder, file)) / 255.0
                i += 1
            else:
                #print(f'test : {i-int(0.8*train_total_size)}')
                dataset_01_test[i-int(0.8*train_total_size)] = cv2.imread(os.path.join(datasets_dir, folder, file)) / 255.0
                i += 1
                
i = 0
for folder in os.listdir(datasets_dir)[:4]:
    if folder.endswith('02'):
        for file in os.listdir(os.path.join(datasets_dir, folder)):
            dataset_02[i] = cv2.imread(os.path.join(datasets_dir, folder, file)) / 255.0
            i += 1

In [42]:
cv2.imshow('',dataset_01_train[700])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [63]:
for folder in os.listdir(datasets_dir):
    if folder.endswith('01'):
        print(folder)
        path = os.path.join(datasets_dir, folder)
        len_folder = len(os.listdir(path))
        len_train = int(0.8*len_folder)
        len_test = int(0.2*len_folder)
        for file in os.listdir(path)[len]:
            print(file)

ernest_celestine_01
ernest-celestine-00033.png
ernest-celestine-00034.png
ernest-celestine-00035.png
ernest-celestine-00036.png
ernest-celestine-00037.png
ernest-celestine-00038.png
ernest-celestine-00039.png
ernest-celestine-00040.png
ernest-celestine-00041.png
ernest-celestine-00042.png
ernest-celestine-00043.png
ernest-celestine-00044.png
ernest-celestine-00045.png
ernest-celestine-00046.png
ernest-celestine-00047.png
ernest-celestine-00048.png
ernest-celestine-00049.png
ernest-celestine-00050.png
ernest-celestine-00051.png
ernest-celestine-00052.png
ernest-celestine-00053.png
ernest-celestine-00054.png
ernest-celestine-00055.png
ernest-celestine-00056.png
ernest-celestine-00057.png
ernest-celestine-00058.png
ernest-celestine-00059.png
ernest-celestine-00060.png
ernest-celestine-00061.png
ernest-celestine-00062.png
ernest-celestine-00063.png
ernest-celestine-00064.png
ernest-celestine-00065.png
ernest-celestine-00066.png
ernest-celestine-00067.png
ernest-celestine-00068.png
ernest-c